In [1]:
import os,sys
from tqdm import tqdm
import geopandas as gpd
import pgpkg
import pygeos
sys.path.append(os.path.join( '..','src'))
from osm_clipper import remove_tiny_shapes,poly_files
import os,sys
from tqdm import tqdm
import geopandas as gpd
import pandas as pd
import numpy as np
from pgpkg import Geopackage
import pygeos
sys.path.append(os.path.join( '..','src'))
from osm_clipper import remove_tiny_shapes,poly_files,planet_osm
from shapely.wkb import loads

In [2]:
data_path = os.path.join('..','data')
glob_ctry_path = os.path.join(data_path,'cleaned_shapes','global_countries.gpkg')

In [3]:
poly_files(data_path,glob_ctry_path)

C:\Users\eks510\.conda\envs\py38\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
# load country file
country_gadm_path = os.path.join(data_path,'GADM36','gadm36_levels.gpkg')
gadm_level0 = pd.DataFrame(gpd.read_file(country_gadm_path,layer='level0'))

#convert to pygeos
gadm_level0['geometry'] = pygeos.from_shapely(gadm_level0.geometry)

# remove antarctica, no roads there anyways
gadm_level0 = gadm_level0.loc[~gadm_level0['NAME_0'].isin(['Antarctica'])]

# remove tiny shapes to reduce size substantially
tqdm.pandas(desc='Remove tiny shapes')
gadm_level0['geometry'] = gadm_level0.progress_apply(remove_tiny_shapes,axis=1)

#simplify geometry
tqdm.pandas(desc='Simplify geometry')
gadm_level0.geometry = gadm_level0.geometry.progress_apply(lambda x: pygeos.simplify(
    pygeos.buffer(
        pygeos.simplify(
            x,tolerance = 0.005, preserve_topology=True),0.01),tolerance = 0.005, preserve_topology=True))  

#save to new country file
glob_ctry_path = os.path.join(data_path,'cleaned_shapes','global_countries.gpkg')

In [ ]:
pygeos.geometry.get_type_id(gadm_level0.geometry).unique()

In [ ]:
with Geopackage(glob_ctry_path, 'w') as out:
    out.add_layer(gadm_level0, name='level0', crs='EPSG:4326')

In [ ]:
gadm_level0.geometry = gadm_level0.geometry.progress_apply(lambda x: loads(pygeos.to_wkb(x)))

In [ ]:
gpd.GeoDataFrame(gadm_level0).to_file(glob_ctry_path,layer='level0', driver="GPKG")

In [ ]:
%%time
gadm_level0['geometry'] = pygeos.from_shapely(gadm_level0.geometry)

In [ ]:
gadm_level0 = gadm_level0.loc[~gadm_level0['NAME_0'].isin(['Antarctica'])]

In [ ]:
def remove_tiny_shapes(x,regionalized=False):
    """This function will remove the small shapes of multipolygons. Will reduce the size of the file.
    
    Arguments:
        *x* : a geometry feature (Polygon) to simplify. Countries which are very large will see larger (unhabitated) islands being removed.
    
    Optional Arguments:
        *regionalized*  : Default is **False**. Set to **True** will use lower threshold settings (default: **False**).
        
    Returns:
        *MultiPolygon* : a shapely geometry MultiPolygon without tiny shapes.
        
    """
    
    # if its a single polygon, just return the polygon geometry
    if pygeos.geometry.get_type_id(x.geometry) == 3: # 'Polygon':
        return x.geometry
    
    # if its a multipolygon, we start trying to simplify and remove shapes if its too big.
    elif pygeos.geometry.get_type_id(x.geometry) == 6: # 'MultiPolygon':
        
        if regionalized == False:
            area1 = 0.1
            area2 = 250
                
        elif regionalized == True:
            area1 = 0.01
            area2 = 50           

        # dont remove shapes if total area is already very small
        if pygeos.area(x.geometry) < area1:
            return x.geometry
        # remove bigger shapes if country is really big

        if x['GID_0'] in ['CHL','IDN']:
            threshold = 0.01
        elif x['GID_0'] in ['RUS','GRL','CAN','USA']:
            if regionalized == True:
                threshold = 0.01
            else:
                threshold = 0.01

        elif pygeos.area(x.geometry) > area2:
            threshold = 0.1
        else:
            threshold = 0.001

        # save remaining polygons as new multipolygon for the specific country
        new_geom = []
        for index_ in range(pygeos.geometry.get_num_geometries(x.geometry)):
            if pygeos.area(pygeos.geometry.get_geometry(x.geometry,index_)) > threshold:
                new_geom.append(pygeos.geometry.get_geometry(x.geometry,index_))
        
        return pygeos.creation.multipolygons(np.array(new_geom))